In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import shared_functions_wlp_wls as shared
import scipy.integrate
import pyccl as ccl

/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/halo_table_cache.py:12: UserWarning: Some of the functionality of the HaloTableCache classrequires h5py to be installed.
  warn("Some of the functionality of the HaloTableCache class"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/user_supplied_ptcl_catalog.py:13: UserWarning: Most of the functionality of the sim_manager sub-package requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Most of the functionality of the sim_manager "
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/download_manager.py:39: UserWarning: Some of the functionality of the DownloadManager requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Some of the functionality of the DownloadManager requires h5py to be installed,\n"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/cached_halo_catalog.py:15: UserWarning: Most of the functionality of the

In [2]:
SURVEY = 'SDSS'
if (SURVEY=='SDSS'):
    import params as pa
elif (SURVEY=='LSST_DESI'):
    import params_LSST_DESI as pa
else:
    print "We don't have support for that survey yet; exiting."
    exit()

Goal: see how well we reproduce the upper panels of Figure 6 from Zu & Mandelbaum 2015 ($w_{gg}$ for different stellar mass bins).

In [3]:
#Initialize the cosmology
OmM = pa.OmC_s + pa.OmB_s; OmB=pa.OmB_s; h0 = pa.HH0_s / 100.; sigma80=pa.sigma8_s; n_s0 = pa.n_s_s;

p = ccl.Parameters(Omega_c = OmM-OmB, Omega_b = OmB, h = h0, sigma8=sigma80, n_s=n_s0)
cosmo = ccl.Cosmology(p)

rho_crit = 3. * 10**10 * pa.mperMpc / (8. * np.pi * pa.Gnewt * pa.Msun)  # Msol h^2 / Mpc^3, for use with M in Msol / h (comoving distances)
rho_m = (OmM) * rho_crit # units of Msol h^2 / Mpc^3 (comoving distances)

In [4]:
# define stellar mass bin limits and z's of bins:
Mslow = np.asarray([8.5, 9.4, 9.8, 10.2, 10.6, 11.0, 11.2, 11.4, 12.])
z = np.asarray([0.02, 0.04, 0.05, 0.08, 0.12, 0.15, 0.17, 0.19])

# Define a halo mass range
lgMh = np.linspace(10., 15., 30)
Mh = 10**(lgMh)

In [5]:
# At each redshift get the halo mass function and halo bias:
HMF = [0]*len(z); bh = [0]*len(z)
for zi in range(0,len(z)):
    HMF[zi]= ccl.massfunction.massfunc(cosmo, Mh / h0, 1./ (1. + z[zi]), odelta=200.) / h0**3
    bh[zi] = ccl.massfunction.halo_bias(cosmo, Mh / h0, 1./(1.+z[zi]), odelta=200.)

Get $<N_{\rm sat}>$ and $<N_{\rm cen}>$ with $M_*$ threshold of each stellar mass bin edge and subtract appropriately to get it in the stellar mass bins plotted in Figure 8.

In [6]:
Nsat_thresh = [0]*len(Mslow)
Ncen_thresh = [0]*len(Mslow)
for mi in range(0,len(Mslow)):
    Nsat_thresh[mi] = shared.get_Nsat_Zu(Mh, 10**Mslow[mi], 'tot', 'SDSS')
    Ncen_thresh[mi] = shared.get_Ncen_Zu(Mh, 10**Mslow[mi], 'SDSS')
    
Nsat_bin = [0] *(len(Mslow) - 1)
Ncen_bin = [0] *(len(Mslow) - 1)
for bi in range(0, len(Mslow)-1):
    Nsat_bin[bi] = Nsat_thresh[bi] - Nsat_thresh[bi+1]
    Ncen_bin[bi] = Ncen_thresh[bi] - Ncen_thresh[bi+1]

Now for each bin get the 1-halo term of the power spectrum.

In [7]:
# First need to get the total number density
nbar = np.zeros(len(z))
for zi in range(0,len(z)):
    nbar[zi] = scipy.integrate.simps(HMF[zi] * (Ncen_bin[zi] + Nsat_bin[zi]), np.log10(Mh / h0))

# Get the pair terms for each bin
Ncensat_bin = [0] * (len(Mslow)-1)
Nsatsat_bin = [0] * (len(Mslow)-1)
for bi in range(0,len(Mslow)-1):
    Ncensat_bin[bi] = Ncen_bin[bi] * Nsat_bin[bi]
    # For the sat-sat case have to make sure we don't accidentally set negative
    Nsatsat_bin[bi]= np.zeros(len(Mh))
    for mi in range(0,len(Mh)):
        if (Nsat_bin[bi][mi]>1.0):
            Nsatsat_bin[bi][mi] = Nsat_bin[bi][mi] * (Nsat_bin[bi][mi] -1.)
        else:
            Nsatsat_bin[bi][mi] = 0.



In [10]:
# Get the Fourier transformed NFW
logkmin = -6; kpts =40000; logkmax = 5; 
kvec_FT = np.logspace(logkmin, logkmax, kpts)
# Actually we will use a downsampled version of this:
k = np.logspace(np.log10(kvec_FT[0]), np.log10(kvec_FT[-1]), 40)
y = shared.gety_ldm(Mh, k, 'SDSS')

In [11]:
# Get the 1-halo power spectrum in each bin
Pkgg_1h = [0]*len(z)
for zi in range(0,len(z)):
    Pkgg_1h[zi] = np.zeros(len(k))
    for ki in range(0,len(k)):
        Pkgg_1h[zi][ki] = scipy.integrate.simps(HMF[zi] * (Ncensat_bin[zi]*y[ki, :] + Nsatsat_bin[zi]*y[ki, :]**2), np.log10(Mh /h0)) / nbar[zi]**2 
        

Now get the 2halo power spectrum in each bin

In [12]:
# Get the linear power spectrum from CCL in each bin
Pklin = [0]*len(z)
for zi in range(0,len(z)):
    # CCL uses units without little-h. Convert to little-h units.
    Pklin[zi] = ccl.power.linear_matter_power(cosmo, k * h0, 1. / (1. + z[zi])) * h0**3

# Get the 2-halo power spectrum in each bin
Pkgg_2h = [0]*len(z)
for zi in range(0,len(z)):
    Pkgg_2h[zi] = np.zeros(len(k))
    for ki in range(0,len(k)):
        Pkgg_2h[zi][ki] = (scipy.integrate.simps(HMF[zi] * bh[zi] * y[ki, :] * (Ncen_bin[zi] + Nsat_bin[zi]), np.log10(Mh /h0)) / nbar[zi])**2 * Pklin[zi][ki]
        

Combine 1 and 2 halo terms and output to file for each bin so we can Fourier transform.

In [19]:
Pgg_tot = [0]*len(z)
for zi in range(0,len(z)):
    Pgg_tot[zi]=np.zeros(len(k))
    for ki in range(0,len(k)):
        if k[ki]>10**(-2):
            Pgg_tot[zi][ki] = Pkgg_2h[zi][ki] + Pkgg_1h[zi][ki]
        else:
            Pgg_tot[zi][ki] = Pkgg_2h[zi][ki]
    
    # This needs to be interms of the more well-sampled k vector
    interp_in_k = scipy.interpolate.interp1d(np.log(k), np.log(Pgg_tot[zi]))
    full_Pgg = np.exp(interp_in_k(np.log(kvec_FT)))
    
    save_P1h2h = np.column_stack((kvec_FT, full_Pgg))
    np.savetxt('./txtfiles/halofit_Pk/Pgg_compFig6_bin='+str(zi)+'.txt', save_P1h2h)

Import $\xi_{gg}(r)$ as Fourier transformed using FFTlog from our power spectrum.

In [31]:
xigg = [0]*len(z)
r = [0]*len(z)
for zi in range(0,len(z)):
    r[zi], xigg[zi] = np.loadtxt('./txtfiles/halofit_xi/xigg_compFig6_bin='+str(zi)+'.txt', unpack=True)
    for ri in range(0,len(r[zi])):
        if xigg[zi][ri]<10**(-12):
            xigg[zi][ri]=10**(-12)

Project to get $w_{gg}$ for each bin.

In [32]:
# For each bin interpolate xi in r
interp_xi = [0]*len(z)
for zi in range(0,len(z)):
    interp_xi[zi] = scipy.interpolate.interp1d(np.log(r[zi]), np.log(xigg[zi]))
    
rp = np.logspace(-2, 2, 100)
Pi = np.logspace(-5, np.log10(80.), 1000) # use the same projection length as Reid & Spergel 2008

# For each bin get xi in terms of rp and Pi
logxi_2d = [0] * len(z)
for zi in range(0,len(z)):
    logxi_2d[zi] = np.zeros((len(rp), len(Pi)))
    for ri in range(0,len(rp)):
        for pi in range(0,len(Pi)):
            r_2d = np.sqrt( rp[ri]**2 + Pi[pi]**2 )
            logxi_2d[zi][ri, pi] = interp_xi[zi](np.log(r_2d))

# Now project at each rp and in each bin            
wgg = [0]*len(z)
for zi in range(0,len(z)):
    wgg[zi] = np.zeros(len(rp))
    for ri in range(0,len(rp)):
        wgg[zi][ri] = 2.*scipy.integrate.simps(np.exp(logxi_2d[zi][ri,:]), Pi)

Now output what we have calculated

In [38]:
plt.figure()
plt.loglog(rp, wgg[0], 'b+')
plt.ylim(5,10000)
plt.xlim(0.03,25)
plt.ylabel('$w_p$')
plt.xlabel('$r_p$')
plt.title('$w_{gg}, M_* = 8.5-9.4$')
plt.savefig('./plots/wgg_bin=0.pdf')
plt.close()

plt.figure()
plt.loglog(rp, wgg[1], 'b+')
plt.ylim(5,10000)
plt.xlim(0.03,25)
plt.ylabel('$w_p$')
plt.xlabel('$r_p$')
plt.title('$w_{gg}, M_* = 9.4-9.8$')
plt.savefig('./plots/wgg_bin=1.pdf')
plt.close()

plt.figure()
plt.loglog(rp, wgg[2], 'b+')
plt.ylim(5,10000)
plt.xlim(0.03,25)
plt.ylabel('$w_p$')
plt.xlabel('$r_p$')
plt.title('$w_{gg}, M_* = 9.8-10.2$')
plt.savefig('./plots/wgg_bin=2.pdf')
plt.close()

plt.figure()
plt.loglog(rp, wgg[3], 'b+')
plt.ylim(5,10000)
plt.xlim(0.03,25)
plt.ylabel('$w_p$')
plt.xlabel('$r_p$')
plt.title('$w_{gg}, M_* = 10.2-10.6$')
plt.savefig('./plots/wgg_bin=3.pdf')
plt.close()

plt.figure()
plt.loglog(rp, wgg[4], 'b+')
plt.ylim(5,10000)
plt.xlim(0.03,25)
plt.ylabel('$w_p$')
plt.xlabel('$r_p$')
plt.title('$w_{gg}, M_* = 10.6-11.0$')
plt.savefig('./plots/wgg_bin=4.pdf')
plt.close()

plt.figure()
plt.loglog(rp, wgg[5], 'b+')
plt.ylim(5,10000)
plt.xlim(0.03,25)
plt.ylabel('$w_p$')
plt.xlabel('$r_p$')
plt.title('$w_{gg}, M_* = 11.0-11.2$')
plt.savefig('./plots/wgg_bin=5.pdf')
plt.close()

plt.figure()
plt.loglog(rp, wgg[6], 'b+')
plt.ylim(5,10000)
plt.xlim(0.03,25)
plt.ylabel('$w_p$')
plt.xlabel('$r_p$')
plt.title('$w_{gg}, M_* = 11.2-11.4$')
plt.savefig('./plots/wgg_bin=6.pdf')
plt.close()

plt.figure()
plt.loglog(rp, wgg[7], 'b+')
plt.ylim(5,10000)
plt.xlim(0.03,25)
plt.ylabel('$w_p$')
plt.xlabel('$r_p$')
plt.title('$w_{gg}, M_* = 11.4-12$')
plt.savefig('./plots/wgg_bin=7.pdf')
plt.close()

Okay so that's all basically correct, up to small factors, which I think are probably explainable by me guessing at redshifts for each bin. That's also not too surprising, given these quantities are largely related to large scales and large scales are not the problem.